In [1]:
import os
os.environ["HF_HOME"] = "E:/huggingface"

#import transformers
from datasets import load_dataset


dataset = load_dataset("squad_it")
DEVICE = "cuda:0"

In [2]:
from get_model import get_model
model_name = "maestrale"

model, tokenizer = get_model(model_name)
#model = model.to(DEVICE) #togliere se in 4 bit

bin c:\Users\Samuele\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from get_prompt import get_prompt
# del generate_prompt
generate_prompt, stop = get_prompt(model_name) #stop sembra useless

In [4]:
def build_question(context, question):
    return f"Dato il seguente testo:\n{context}\nRispondi brevemente a questa domanda:\n{question}"

def build_answer(answer):
    return f"Risposta breve: {answer}"

In [5]:
import random

def get_shots(dataset, n):
    conversation = []
    for i in range(n):
        elem = random.choice(dataset["train"])
        q_shot = build_question(elem["context"], elem["question"])
        conversation.append(dict(
            role="user",
            text=q_shot
        ))
        conversation.append(dict(
            role="assistant",
            text=build_answer(elem["answers"]["text"][0])
        ))
    return conversation

In [6]:
chat = [
  {"role": "user", "content": "SHOT 1" },
  {"role": "assistant", "content": "ANSWER 1"},
  {"role": "user", "content": "SHOT 2"},
  {"role": "assistant", "content": "ANSWER 2"},
  {"role": "user", "content": "SHOT 3"},
  {"role": "assistant", "content": "ANSWER 3"},
  {"role": "user", "content": "DOMANDA"},
]
 
print(tokenizer.apply_chat_template(chat, tokenize=False))

<|im_start|>user
SHOT 1<|im_end|>
<|im_start|>assistant
ANSWER 1<|im_end|>
<|im_start|>user
SHOT 2<|im_end|>
<|im_start|>assistant
ANSWER 2<|im_end|>
<|im_start|>user
SHOT 3<|im_end|>
<|im_start|>assistant
ANSWER 3<|im_end|>
<|im_start|>user
DOMANDA<|im_end|>



In [7]:
import torch

def forward_model(*, prompt, model, tokenizer):
    if "pad_token" not in tokenizer.special_tokens_map:
        tokenizer.pad_token = tokenizer.eos_token

    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        padding=True,
    ).input_ids
    with torch.no_grad():
        input_ids = input_ids.to(DEVICE)
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=32,
            pad_token_id=tokenizer.eos_token_id,
            prompt_lookup_num_tokens=10,
            temperature = 1
        )
        # print(output_ids)

    ret = []
    for i in range(0, len(output_ids)):
        generated_text = tokenizer.decode(
            output_ids[i],
            skip_special_tokens=True
        )
        #print(generated_text[len(prompt[i]):])
        generated_text = generated_text[len(prompt[i]):] #perchè in output_ids c'è anche il prompt!

        # if "\n" in generated_text:
        #     generated_text = generated_text[:generated_text.index("\n")] non necessario
        
        ret.append(generated_text.strip())

    return ret

In [8]:
import time

t0 = time.time()
out = forward_model(
    prompt=["ciao amico"],
    model=model,
    tokenizer=tokenizer,
)
print(time.time() - t0)

print(out)

4.532660007476807
['ciao amico\n\nciao amico\n\nciao amico\n\nciao amico\n\nciao amico']


In [ ]:
# def generate_prompt(conversation, do_continue=False):
#     # prompt = """[INST]<<SYS>>\nSei un assistente disponibile, rispettoso e onesto. Rispondi sempre nel modo piu' utile possibile, pur essendo sicuro. Le risposte non devono includere contenuti dannosi, non etici, razzisti, sessisti, tossici, pericolosi o illegali. Assicurati che le tue risposte siano socialmente imparziali e positive. Se una domanda non ha senso o non e' coerente con i fatti, spiegane il motivo invece di rispondere in modo non corretto. Se non conosci la risposta a una domanda, non condividere informazioni false.\n<</SYS>>\n\n"""
#     assert conversation[-1]['role'] == "user"
#     ST, ET = "<s>", "</s>" #Start/End Text
#     SUM, EUM = "[INST]", "[/INST]" #Start/End User Message
#     prompt = ""
#     for message in conversation:
#         if message['role'] == "user":
#             prompt += f"{ST}{SUM}{message['text']}{EUM}"
#         elif message['role'] == "assistant":
#             prompt += f"{message['text']}{ET}"
#         else:
#             raise ValueError(f"Role not found,\nFounded {message['role']}")
    

#     if do_continue:
#         prompt += "\n" # non mi convince
    
#     return prompt

In [9]:
### CELLA DI PROVA DEL NUOVO GET PROMPT DI SAIGA-ITA-7B!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import uuid
from tqdm import tqdm

batch_size = 1

predicted_answers = []
theoretical_answers = []

ds = dataset["test"].shuffle(seed=42)

total = 0
for i in range(3):
    elem     = ds[i].copy()
    ids      = elem["id"]
    context  = elem["context"]
    question = elem["question"]
    answers  = elem["answers"]

    model_inputs = []
    chat = get_shots(dataset, 3) + [
                dict(
                    role="user",
                    text=build_question(context, question)
                )
            ]
    model_inputs.append(
        generate_prompt(
        # scrive tre esempi non come  prompt ma come cronologia di chat: role: user -> text: domanda, role: assistant -> text: risposta breve
            chat,
            do_continue=False
        ) + " " + build_answer("").strip()
    )

print(model_inputs[0])

<|im_start|>user
Dato il seguente testo:
Il sistema di transito rapido ferroviario Staten Island Railway serve unicamente Staten Island, operativo 24 ore su 24. L' Autorità Portuale Trans-Hudson (treno PATH) collega Midtown e Lower Manhattan al New Jersey nord-orientale, principalmente Hoboken, Jersey City e Newark. Come la metropolitana di New York City, la PATH opera 24 ore su 24; ciò significa che tre dei sei sistemi di transito rapido nel mondo che operano su orari di 24 ore sono totalmente o parzialmente a New York (gli altri sono una parte della Chicago' L', la PATCO Speedline che serve Filadelfia e la metropolitana di Copenaghen).
Rispondi brevemente a questa domanda:
Quanti sistemi di transito rapido 24 ore su 24 si trovano a New York?<|im_end|>
<|im_start|>assistant
Risposta breve: tre<|im_end|>
<|im_start|>user
Dato il seguente testo:
Il massiccio del Vinson, la vetta più alta dell' Antartide a 4.892 m, si trova nelle montagne dell' Ellsworth. L' Antartide contiene molte altr

In [10]:
import uuid
from tqdm import tqdm

batch_size = 1

predicted_answers = []
theoretical_answers = []

ds = dataset["test"].shuffle(seed=42)

bar = tqdm(ds.iter(batch_size=batch_size), total=len(ds))
total = 0
for elem in bar:
    try:
        ids = elem["id"]
        context = elem["context"]
        question = elem["question"]
        answers = elem["answers"]

        model_inputs = []
        for c, q in zip(context, question):
            model_inputs.append(
                generate_prompt(
                    get_shots(dataset, 3) + [
                        dict(
                            role="user",
                            text=build_question(c, q)
                        )
                    ],
                    do_continue=True
                ) + " " + build_answer("").strip()
            )
        # print(model_inputs[0])
        # break

        model_outputs = forward_model(
            prompt=model_inputs[0],
            model=model,
            tokenizer=tokenizer,
        )
        model_outputs = model_outputs[0].split("Risposta breve:")[-1] if "Risposta breve:" in model_outputs[0] else _
        print(f"DOMANDA:\n{model_inputs[0]}\n\nRISPOSTA\n{model_outputs}\nSOLUZIONE\n{answers[0]['text']}\n")#[520:]

        # for model_output, id in zip(model_outputs, ids):
        #     predicted_answers.append(dict(
        #         id=id,
        #         prediction_text=model_output,
        #     ))
        predicted_answers.append(dict(
                id=ids[0],
                prediction_text=model_outputs,
            ))

        for ans, id in zip(answers, ids):
            theoretical_answers.append(dict(
                id=id,
                answers=ans,
            ))
        
    except Exception as e:
        print(e)
        continue

  0%|          | 1/7609 [00:08<18:24:43,  8.71s/it]

DOMANDA:
<|im_start|>Dato il seguente testo:
Nel dicembre 2006, Robert "Evel" Knievel ha citato in giudizio West per violazione del marchio nel video di West per "Touch the Sky". Knievel ha preso il problema con un "video sessualmente carico" in cui West assume la persona di "Evel Kanyevel" e tenta di volare un razzo sopra un canyon. La causa ha denunciato la violazione del nome e della somiglianza marcata di Knievel. Knievel ha anche sostenuto che le immagini "volgari e offensive" raffigurate nel video danneggiavano la sua reputazione. La causa ha chiesto il risarcimento del danno pecuniario e un' ingiunzione per interrompere la distribuzione del video. Avvocati West's ha sostenuto che il video musicale era pari a satira e quindi è stato coperto dal primo emendamento. Pochi giorni prima della sua morte, nel novembre 2007, Knievel ha trovato un accordo amichevole dopo aver ricevuto una visita dall' Occidente, dicendo:"Pensavo che fosse un ragazzo meraviglioso e un gentiluomo".
Rispondi

  0%|          | 2/7609 [00:14<14:59:07,  7.09s/it]

DOMANDA:
<|im_start|>Dato il seguente testo:
Il contemporaneo adulto affonda le sue radici nel formato easy listening degli anni' 60, che ha adottato un mix vocale strumentale 70-80% - 20-30%. Pochi offrivano il 90% degli strumenti e pochi strumenti erano del tutto strumentali. Il formato di facile ascolto, come era noto per la prima volta, è nato dal desiderio di alcune emittenti radiofoniche tra la fine degli anni Cinquanta e l' inizio degli anni Sessanta di continuare a suonare canzoni di successo, ma distinguersi dall' essere marchiate come stazioni "rock and roll". Billboard ha pubblicato per la prima volta il grafico Easy Listening il 17 luglio 1961, con 20 canzoni; il primo numero uno era "Boll Weevil Song" di Brook Benton. Il grafico si descriveva come "non troppo lontano in entrambe le direzioni".
Rispondi brevemente a questa domanda:
In quale data è stato pubblicato per la prima volta il diagramma Billboard easy listening?<|im_end|> Risposta breve: 17 luglio 1961<|im_start|>D

  0%|          | 3/7609 [00:17<11:15:46,  5.33s/it]

...

DOMANDA:
<|im_start|>Dato il seguente testo:
Per le finali, American Idol ha debuttato l' 11 marzo 2008 con un nuovo set e palcoscenico all' avanguardia, insieme ad un nuovo look in onda. L' interpretazione di David Cook di "Billie Jean" nella top-ten night è stata lodata dai giudici, ma ha suscitato polemiche quando apparentemente hanno confuso l' arrangiamento di Chris Cornell per essere David Cook, anche se l' esecuzione è stata introdotta come versione di Cornell. Cornell stesso ha detto di essere "lusingato" e ha lodato la performance di David Cook. David Cook è stato portato in ospedale dopo lo spettacolo di prestazioni top-nine grazie alle palpitazioni cardiache e all' alta pressione sanguigna.
Rispondi brevemente a questa domanda:
Quale arrangiamento è stato utilizzato per la versione di Billie Jean di Cook?<|im_end|> Risposta breve: Chris Cornell<|im_start|>Dato il seguente testo:
Il 20 dicembre 2005, la Corte d' Appello degli Stati Uniti per il Sesto Circuito ha stabilito nel

  0%|          | 24/7609 [01:55<8:35:25,  4.08s/it]

DOMANDA:
<|im_start|>Dato il seguente testo:
Nel maggio 1940, gli Stati Uniti chiesero al Regno Unito i diritti di base alle Bermuda, ma il Primo Ministro britannico Winston Churchill inizialmente non era disposto ad aderire alla richiesta americana senza ricevere alcun compenso. Nel settembre 1940, nel quadro dell' accordo sui distruttori delle basi, il Regno Unito concedeva i diritti di base statunitensi alle Bermuda. Bermuda e Terranova non erano originariamente incluse nell' accordo, ma sono state entrambe aggiunte, senza che il Regno Unito abbia ricevuto in cambio materiale bellico. Uno dei termini dell' accordo era che l' aeroporto costruito dall' esercito statunitense sarebbe stato utilizzato congiuntamente dagli Stati Uniti e dal Regno Unito (che per tutta la durata della guerra, con il trasferimento del comando di trasporto della RAF dall' isola di Darrell nel 1943).
Rispondi brevemente a questa domanda:
Chi era il Primo Ministro che ha raggiunto questo accordo con gli Stati U

  0%|          | 24/7609 [01:56<10:13:39,  4.85s/it]


KeyboardInterrupt: 

In [ ]:
print(model_inputs[0])

In [ ]:
model_outputs

In [ ]:
predicted_answers[-1]
#theoretical_answers[0]

In [ ]:
import os
import json


# Create the directory if it doesn't exist
os.makedirs('./cache', exist_ok=True)
with open(f"./cache/generated-squad-{model_name}.json", "w") as f:
    json.dump(dict(
        predicted_answers=predicted_answers,
        theoretical_answers=theoretical_answers,
    ), f, indent=4)

In [ ]:
import json
with open(f"./cache/generated-squad-{model_name}.json", "r") as f:
    data = json.load(f)

print(len(data["predicted_answers"]))

In [ ]:
import evaluate

predicted_answers = data["predicted_answers"]
theoretical_answers = data["theoretical_answers"]

metric = evaluate.load("squad")
results = metric.compute(predictions=predicted_answers, references=theoretical_answers)

import time


print("=== REPORT ===")
print("current date:", time.strftime("%d/%m/%Y %H:%M:%S"))
print("Dataset: SQuAD-it")
print("Model:", model_name)
print(results)
print("==========================")